In [1]:
import torch
import os
import optuna
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Subset, random_split, ConcatDataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

import cv2

/home/jose/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-23 20:21:27.498666: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 20:21:29.252267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
"""
-- Normalize pixel values and create training and testing datasets --

This cell normalizes the pixel values and creates a training
and testing datasets considering the stratify technique.
"""

parent_folder_path = 'images/cropped_dataset/'
categories = {'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral_Pneumonia': 3}
arrays = []
category_amount = []

# get the category with the least images
for category in categories.keys():
    folder_path = os.path.join(parent_folder_path, category)
    image_files = os.listdir(folder_path)
    category_amount.append(len(image_files))

max_training = min(category_amount)

# convert the images into a pytorch dataset
for cat_folder, value in categories.items():

    folder_path = os.path.join(parent_folder_path, cat_folder)
    image_files = os.listdir(folder_path)

    for i, file_name in enumerate(image_files):

        if i >= max_training: break
        file_path = os.path.join(folder_path, file_name)
        image = Image.open(file_path)
        image_array = np.array(image)

        # verify all images are of the desired size
        if image.size != (250, 250):
            print(file_path, " IS NOT 250x250, it is: ", image.size)
            continue

        if image_array.shape != (250, 250):
            image_array = np.dot(image_array[..., :3], [0.2989, 0.5870, 0.1140])

        arrays.append(image_array)

# reshape the array
arrays = np.array(arrays).astype(np.float32)
arrays = arrays/ np.max(arrays)
#image_data = np.stack(arrays, axis=0)
#image_data = image_data.reshape(len(image_data), 1, 250, 250)

arrays_labels = [0] * max_training
arrays_labels += [1] * max_training
arrays_labels += [2] * max_training
arrays_labels += [3] * max_training

arrays_labels = np.array(arrays_labels)

X_train, X_test, y_train, y_test = train_test_split(arrays, arrays_labels, test_size=0.2, random_state=42, stratify=arrays_labels)

print("Finished")

y_train = to_categorical(y_train.astype(int), num_classes=4)
y_test = to_categorical(y_test.astype(int), num_classes=4)

Finished


In [3]:
# Modelo de red MLP personalizado
model_mlp = Sequential()
model_mlp.add(Flatten(input_shape=(250, 250, 1)))

# Capa oculta 1
model_mlp.add(Dense(64, activation='relu'))
model_mlp.add(Dropout(0.2))  # Se agrega Dropout para regularización

# Capa oculta 2
model_mlp.add(Dense(32, activation='relu'))
model_mlp.add(Dropout(0.2))  # Se agrega Dropout para regularización

# Capa de salida
model_mlp.add(Dense(4, activation='softmax'))

# Compilar el modelo
optimizer = Adam(learning_rate=0.01)
model_mlp.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Entrenar el modelo
model_mlp.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1)

2023-05-23 20:22:03.221572: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2285500000 exceeds 10% of free system memory.


Epoch 1/10
143/143 [==============================] - 8s 47ms/step - loss: 12.8577 - accuracy: 0.2635
Epoch 2/10
143/143 [==============================] - 7s 46ms/step - loss: 1.2397 - accuracy: 0.3916
Epoch 3/10
143/143 [==============================] - 7s 46ms/step - loss: 1.1580 - accuracy: 0.4161
Epoch 4/10
143/143 [==============================] - 7s 46ms/step - loss: 1.1753 - accuracy: 0.4078
Epoch 5/10
143/143 [==============================] - 7s 47ms/step - loss: 1.1791 - accuracy: 0.4094
Epoch 6/10
143/143 [==============================] - 7s 46ms/step - loss: 1.1406 - accuracy: 0.4240
Epoch 7/10
143/143 [==============================] - 7s 49ms/step - loss: 1.1475 - accuracy: 0.4173
Epoch 8/10
143/143 [==============================] - 7s 49ms/step - loss: 1.1288 - accuracy: 0.4286
Epoch 9/10
143/143 [==============================] - 7s 47ms/step - loss: 1.1435 - accuracy: 0.4182
Epoch 10/10
143/143 [==============================] - 7s 46ms/step - loss: 1.1428 - accur

In [4]:
pred = model_mlp.predict(X_test)
print(pred)

 1/72 [..............................] - ETA: 9s

2023-05-23 20:23:13.115326: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 571500000 exceeds 10% of free system memory.


72/72 [==============================] - 1s 12ms/step
[[0.30952471 0.3097772  0.2868947  0.09380347]
 [0.30952471 0.3097772  0.2868947  0.09380347]
 [0.30952471 0.3097772  0.2868947  0.09380347]
 ...
 [0.3095247  0.30977717 0.28689468 0.09380345]
 [0.02987056 0.04122603 0.24392258 0.6849808 ]
 [0.3095247  0.30977717 0.28689468 0.09380345]]
